#Interpreting Hugging Face transformers with SHAP

Copyright 2023, Denis Rothman, MIT License

# Introducing Shap

In [1]:
#SHAP with an Iterator
import itertools

# Define the base scores of the words
words = {'I': 0.2, 'love': 0.6, 'playing': 0.5, 'chess': 0.4, 'with': 0.1, 'my': 0.3, 'friends': 0.4}

# Define the bonus scores for certain combinations of words
bonus = {('I', 'love'): 0.3, ('love', 'playing'): 0.25, ('playing', 'chess'): 0.45, ('with', 'my', 'friends'): 0.35}

# Function to calculate the total score of a coalition
def total_score(coalition):
    score = sum(words[word] for word in coalition)
    for b in bonus.keys():
        if all(word in coalition for word in b):
            score += bonus[b]
    return score

# Function to calculate the Shapley value of a word
def shapley_value(word):
    N = len(words)
    permutations = list(itertools.permutations(words))
    marginal_contributions = []
    counter = 0  # Counter initialization
    for permutation in permutations:
        index = permutation.index(word)
        coalition_without_word = permutation[:index]
        coalition_with_word = permutation[:index+1]
        marginal_contribution = total_score(coalition_with_word) - total_score(coalition_without_word)
        marginal_contributions.append(marginal_contribution)
        counter += 1  # Increment counter
    print(f"Processed {counter} permutations")  # Print counter
    return sum(marginal_contributions)

# Calculate the Shapley value of each word
for word in words:
    print(f"The Shapley value of '{word}' is {shapley_value(word)}")

Processed 5040 permutations
The Shapley value of 'I' is 1764.0000000000798
Processed 5040 permutations
The Shapley value of 'love' is 4409.99999999998
Processed 5040 permutations
The Shapley value of 'playing' is 4283.999999999768
Processed 5040 permutations
The Shapley value of 'chess' is 3150.000000000016
Processed 5040 permutations
The Shapley value of 'with' is 1092.000000000074
Processed 5040 permutations
The Shapley value of 'my' is 2099.999999999982
Processed 5040 permutations
The Shapley value of 'friends' is 2604.0000000001683


# Explaining Hugging Face outputs with SHAP

Install packages

In [ ]:
#Hugging Face Transformers
!pip install transformers
#Transformer building blocks
!pip install xformers
#SHAP
!pip install shap

In [19]:
#@title Enter your sentence here:
sentence = 'SHAP is a useful explainer' #@param {type:"string"}

Sentiment Analysis

In [20]:
import transformers

# load a transformers pipeline model
model = transformers.pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

# analyze the sentiment of the input sentence
result = model(sentence)[0]
print(result)

{'label': 'POSITIVE', 'score': 0.9869391918182373}


In [21]:
import shap
# explain the model on the input sentence
explainer = shap.Explainer(model) 
shap_values = explainer([sentence])

# visualize the first prediction's explanation for the predicted class
predicted_class = result['label']
shap.plots.text(shap_values[0, :, predicted_class])

In [22]:
# get the sentiment score 
sentiment_score = model(sentence)[0]['score']

# print the SHAP values for each word
words = sentence.split(' ')
for word, shap_value in zip(words, shap_values.values[0, :, 0]):
    print(f"Word: {word}, SHAP value: {shap_value}")

Word: SHAP, SHAP value: 0.0
Word: is, SHAP value: 0.12446051090955734
Word: a, SHAP value: 0.17050934582948685
Word: useful, SHAP value: -0.11854982748627663
Word: explainer, SHAP value: -0.09484541043639183
